# Finding AOM Delay

1. Set the AOM delay to 0 in the hardware interfaces to use this code
2. Run the file that updates the experimental configuration
3. Run an image scan and track on a very bright point object. If it is an NV keep in mind the 12ns response time to subtract from the fit time
4. Run the code below and generate the graph

In [ ]:
from experimental_configuration import *
import sys
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

from NV_ABJ.experimental_logic.sequence_generation.sequence_generation import *
from NV_ABJ.abstract_interfaces.photon_counter import PhotonCounter
from NV_ABJ.abstract_interfaces.pulse_generator import PulseGenerator
import numpy as np
import nidaqmx
from nidaqmx.constants import CountDirection, Edge

class AomDelayMeasurement:

    def get_counts(self, photon_counter:PhotonCounter,
                              pulse_generator:PulseGenerator,
                              aom_trigger:SequenceDevice, 
                              readout_trigger:SequenceDevice,
                              maximum_readout_time_ns:float,
                              minimum_readout_time_ns:float = 410,
                              minimum_pulse_time_ns:int=10,
                              window_size_ns:float=400,
                              readout_trigger_time_ns:float=25,
                              reset_time_ns:float = 1000,
                              averaging_cycles:int = 20_000)->np.typing.NDArray:
        
        # Generating Sequence and starting pulse generator 
        
        counts_per_time = []
        counts_error = []
        measured_times = []

        time_offsets = np.arange(minimum_readout_time_ns,maximum_readout_time_ns, minimum_pulse_time_ns)

        for it,time_offset in enumerate(time_offsets):

            seq = Sequence()

            if time_offset>window_size_ns and time_offset-window_size_ns>=minimum_pulse_time_ns:
                
                seq.add_step(window_size_ns,[aom_trigger])
                seq.add_step(time_offset-window_size_ns,[])
                seq.add_step(window_size_ns-readout_trigger_time_ns,[readout_trigger])
                seq.add_step(window_size_ns,[])
                seq.add_step(readout_trigger_time_ns,[readout_trigger])

            else:
                # Skips if the offset is less than the minimum pulse time
                print(time_offset)
                continue

            measured_times.append(time_offset)

            # This is a reset for the delay it just needs to be longer than the expected aom delay
            seq.add_step(reset_time_ns,[])
          

            pulse_generator.stop()
            seq_text = pulse_generator.generate_sequence(seq)

            # print(seq_text)

            pulse_generator.load(sequence=seq_text)
            pulse_generator.start()

            # Getting the sample of counts
            with photon_counter as pc:
            
                counts = np.array(pc.get_counts_raw_when_triggered(number_of_data_taking_cycles=averaging_cycles))

            counts_per_time.append(np.mean(counts))
            counts_error.append(np.std(counts))
            
            print(f"Complete: {round((it+1)/len(time_offsets)*100)}%",end="\r")

        # returns times and counts 
        return np.array(measured_times), np.array(counts_per_time), np.array(counts_error)



aom_measure = AomDelayMeasurement()

time_offsets,counts, error_counts = aom_measure.get_counts(photon_counter=apd_1,
                                        pulse_generator=pulse_blaster,
                                        aom_trigger=green_aom_trigger,
                                        readout_trigger=apd_trigger_1,
                                        maximum_readout_time_ns=1400,
                                        reset_time_ns=50000,
                                        minimum_readout_time_ns=410,
                                        window_size_ns=400,
                                        averaging_cycles=50_000)

plt.scatter(time_offsets,counts)
plt.xlabel("Time(ns)")
plt.ylabel("Average Counts")
plt.title(f"AOM Delay")
plt.show()

# Finding the AOM Delay
You should be presented below with a plot that has a clear peak somewhat like a rounded off triangle. Your AOM delay is the time is the time it takes to get to the peak. You can do some fitting and it can look similar to the plot below 

![alt text](download-1.png)

If the plot looks more like the following you have to reset the AOM delay in you experimental_configuration package to 0 and run the file that updates the experimental configuration  

![alt text](download-2.png)

# Sloppy Code For Fitting 

In [ ]:
leading_times = time_offsets[35:55] # Indexes that are the first linear part sloping up
leading_counts = counts[35:55] 
falling_times = time_offsets[70:90] # Indexes for the second linear part sloping down
falling_counts = counts[70:90]

def linear_func(x,m,b):
    return x*m+b

leading_param, leading_conv = curve_fit(linear_func, leading_times, leading_counts)
falling_param, falling_conv = curve_fit(linear_func,falling_times, falling_counts)

t_intersect = (leading_param[1]-falling_param[1])/(falling_param[0]-leading_param[0])
cnt_intersect = leading_param[0]*t_intersect+leading_param[1]

leading_line_times = np.linspace(min(leading_times),t_intersect,200)
falling_line_times = np.linspace(t_intersect,max(falling_times),200)

leading_line_counts = leading_param[0]*leading_line_times+leading_param[1]
falling_line_counts =falling_param[0]*falling_line_times+falling_param[1]

plt.errorbar(time_offsets,counts,yerr=error_counts/np.sqrt(50_000), color = "peachpuff", ecolor="coral")
plt.vlines([t_intersect],ymin=min(counts),ymax=cnt_intersect, color="gray",linestyle="--")
plt.plot(leading_line_times, leading_line_counts,color="blue",alpha=0.2)
plt.plot(falling_line_times,falling_line_counts,color = "blue",alpha=0.2)
plt.xlabel("Time(ns)")
plt.ylabel("Average Counts")
plt.title(f"AOM Delay: {round(t_intersect,2)}ns")
plt.show()